In [1]:
import pandas as pd
import math

In [2]:
from datetime import datetime
date = datetime.fromtimestamp(1584148400	).date()
print(date)

2020-03-13


In [17]:
main = pd.read_excel("statePop.xlsx")
AL_data = pd.read_excel("States_data/AL_data.xlsx")

In [18]:
R_0 = 2                                # Basic reproduction ratio
incubationDur = 6.4                    # Average incubation per
infectionDur = 7                       # Average dur of infection
dt = 1                                 # Time step
peopleSeen = 10                        # Avg number of people seen by average American
asymp = .4                             # (1-asymp) is prob of asympotomatic

alpha = 1 - math.exp(-1/incubationDur) # Daily prob of individual becoming infectious
gamma = 1 - math.exp(-1/infectionDur)  # Daily prob an individual recovers
beta = R_0 * gamma                     # Scaled transmission rate

In [20]:
def numSus(t, pop): # Susceptible to Exposed func
    infProb = I[t] * peopleSeen * pop/S[i]
    return infProb * dt # How many people infected people have met with, weighted by population outdoors

def numInfected(t): 
    return (1-asymp) * alpha * E[t] * dt # daily prob of being infected after exposure * exposed pop

def numRecovery(t):
    return gamma * I[t] * dt # daily prob that an individual recovers * number of infected people

In [21]:
def sGrowth(t, pop): # Current pop (S[t]) - exposed
    return S[t] - numSus(t, pop)

def eGrowth(t, pop): # number of people who have been exposed (numSus) - number of people infected in last timestep
    return numSus(t, pop) - numInfected(t)

def iGrowth(t): # Infected previously + newly infected - recovered + growth in actual infected cases
    return I[t] + numInfected(t) - numRecovery(t) 

def rGrowth(t): # Recovered
    return R[t] + numRecovery(t)

In [22]:
num = 23
S = [main['Population'].iloc[0]]
E = [0]
I = [0]
R = [0]
timeIndex = 0
firstCase = False

for i in range(num):
    if (i < 2):
        datePath = '3-0' + str(i+8)
    else:
        datePath = '3-' + str(i+8)
    dateNow = datePath + '-2020'
    ourDate = datetime.strptime(dateNow, '%m-%d-%Y').date()
    
    if (True):
        infectedNum = 0
        for j in range(timeIndex, AL_data.shape[0]):
            covidDate = AL_data['seconds_since_Epoch'].iloc[j]
            covidDate = datetime.fromtimestamp(covidDate).date()
            if (ourDate == covidDate):
                infectedNum = int(AL_data.iloc[j][2])
                if (infectedNum != 0):
                    firstCase = True
            elif (covidDate > ourDate):
                timeIndex = j
                break
        print(infectedNum)
        if (infectedNum == 6):
            I[i] += infectedNum
    
    path = "data/data" + datePath + '.xlsx'
    data = pd.read_excel(path)
    percentAtHome = data[data['stname'] == 'Alabama']['percentAtHome'][0]
    atHome = percentAtHome/100 * S[i]
    
    S.append(sGrowth(i, S[i] - atHome))
    E.append(eGrowth(i, S[i] - atHome))
    I.append(iGrowth(i))
    R.append(rGrowth(i))
    print("Iteration: " + ourDate.strftime("%m/%d/%Y"))
    print("Susceptible: ", end="")
    print(S)
    print("Exposed: ", end="")
    print(E)
    print("Infected: ", end="")
    print(I)
    print("Recovered: ", end="")
    print(R)
    print("\n")

0
Iteration: 03/08/2020
Susceptible: [4903185, 4903185.0]
Exposed: [0, 0.0]
Infected: [0, 0.0]
Recovered: [0, 0.0]


0
Iteration: 03/09/2020
Susceptible: [4903185, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0]
Infected: [0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0]


0
Iteration: 03/10/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0]


0
Iteration: 03/11/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0, 0.0]


0
Iteration: 03/12/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0.0]
Infected: [0, 0.0, 0.0, 0.0, 0.0, 0.0]
Recovered: [0, 0.0, 0.0, 0.0, 0.0, 0.0]


6
Iteration: 03/13/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903133.509552802]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0


720
Iteration: 03/28/2020
Susceptible: [4903185, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903185.0, 4903133.509552802, 4903092.182405101, 4903025.405126058, 4902938.649359011, 4902822.516508993, 4902668.193388959, 4902463.865603192, 4902199.760122826, 4901886.609576585, 4901473.598844523, 4900915.178924581, 4900176.194986803, 4899167.707841854, 4897846.965143346, 4896108.231491038, 4893917.523953389]
Exposed: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 51.49044719810359, 36.85814742938364, 63.57825709208558, 81.23763186466141, 109.08200818951208, 144.855586718981, 191.75536327660714, 247.46249477857427, 291.67258252958504, 387.695650886244, 524.7707274520359, 693.4376150863382, 948.3017502039245, 1238.4369309340843, 1631.2462389720074, 2049.1271031802903]
Infected: [0, 0.0, 0.0, 0.0, 0.0, 6.0, 5.2012673985010895, 8.977864030193386, 10.981733866208096, 15.037957571566025, 20.08691490479312, 26.88043591990943, 35.87447832533571, 47.74177801239477, 62.86425596525416, 79.8107153560469, 102.835337795